In [98]:
import pandas as pd

In [99]:
df = pd.read_excel("data_cleaned_1.xlsx")

# Chuẩn hoá dữ liệu

> Làm gọn tên sản phẩm

***Chuột***

In [100]:
#Chuột

***Loa***

In [101]:
#Loa

***Sạc dự phòng***

In [102]:
#Sạc dự phòng

In [103]:
mask_powerbank = df['category'].str.contains('sạc dự phòng', case=False, na=False)

In [118]:
name_map = {
    '[DUNG LƯỢNG 30000MAH] SẠC DỰ PHÒNG MẶT GƯƠNG DUNG LƯỢNG 300000MAH - PIN SẠC DỰ PHÒNG 30000 MAH, HỖ TRỢ SẠC NHIỀU MÁY CÙNG LÚC, CÓ ĐÈN LED CHIẾU SÁNG THÔNG MINH': 'Mặt gương 30000 mah',
    '[FREESHIP TẶNG CÁP SẠC NHANH 39K] PIN DỰ PHÒNG LCD MẶT GƯƠNG DUNG LƯỢNG 30000MAH - SẠC DỰ PHÒNG CÓ ĐÈN LED HIỂN THỊ DUNG LƯỢNG PIN - HỖ TRỢ SẠC NHANH ]': 'Mặt gương LCD 30000 mah',
    '[SẠC MỌI DÒNG ĐIỆN THOẠI] PIN SẠC DỰ PHÒNG 20000MAH ĐI KÈM VỚI BỐN DÂY CÓ THỂ THÁO RỜI, VỚI ĐÈN LED, PIN LITHIUM-ION POLYMER, HỔ TRỢ SẠC NHANH, AN TOÀN TUYỆT ĐỐI': 'LITHIUM-ION POLYMER 20000 mah',
    'Lacepulse 丨 mini pin sạc dự phòng với bốn được xây dựng trong cáp, 20000mAh dung tích lớn đồ sạc di động' : 'Lacepulse 20000 mah',
    'Sạc Dự Phòng Từ Tính Không Dây Đa Năng 10000mAh WEKOME WP-47 - Hỗ Trợ Sạc Nhanh PD 20W, Sạc Không Dây Từ Tính Max 15W - Hàng Chính Hãng': 'WEKOME WP-47 10000 mah',
    'Pin Sạc Dự Phòng Philips DLP6080WT 8000mAh' : 'Philips DLP6080WT 8000mAh',
    'Xmobile 100W M3 tích hợp đèn Pin': 'Xmobile M3',
    'Xmobile 22.5W DX367 tích hợp Adapter sạc' : 'Xmobile DX367',
    'Xmobile 22.5W JP312 kèm Cáp Lightning và Type CMẫu mới' : 'Xmobile JP312',
    'Xmobile 55W JP326' : 'Xmobile JP326',
    'Pin Sạc Dự Phòng Anker PowerCore 13000mAh - A1215021': 'Anker PowerCore 13000mAh',
    'Pin sạc dự phòng ROCKSPACE Y30 10000mah sạc nhanh 22,5W tích hợp củ sạc, hàng chính hãng bảo hành 12 tháng ': 'ROCKSPACE Y30 10000mah',
    'Sạc Dự Phòng Xiaomi Power Bank 20000mAh (Integrated Cable) TH | Sạc Nhanh | Phù Hợp Đi Máy Bay | Hàng Chính Hãng': 'Xiaomi TH  20000mAh',
    'Sạc Dự Phòng trong suốt 30000mAh - sạc siêu nhanh, tích hợp 4 cáp, 2 cổng usb, sạc cùng lúc nhiều điện thoại - Hàng chính hãng': 'trong suốt 30000mAh',
    'Pin sạc dự phòng 10000mAh Type C 22.5W Xiaomi Lite - GiaPhucStore | Hàng Chính Hãng': 'Xiaomi Lite 10000mAh',
    'Pin Sạc dự phòng không dây nam châm 5-in-1 ROCK P50 10000mAh 20W tích hợp chân đế giữ điện thoại - Hàng chính hãng bảo hành 12 tháng': 'ROCK P50 10000mAh',
    '[Mỹ] Sạc dự phòng Energizer QM10001AC Thông minh Sạc nam châm từ tính Dung lượng 10000mAh - Hàng chính hãng ': 'Energizer QM10001AC 10000mah',
    '[Mỹ] Sạc dự phòng Energizer QM10002PQ Sạc nam châm từ tính Dung lượng 10000mAh tích hợp chân đế xếp gọn - Hàng chính hãng':'Energizer QM10002PQ 10000mah',
    'Sạc dự phòng 9FIT Pro 20,000mAh /3.7V Li-Polymer - 9PP20K04B, hỗ trợ sạc nhanh - Hàng chính hãng': '9FIT Pro 20000mAh',
    'Sạc dự phòng Energizer 10,000mAh /3.7V Li-Polymer UE10050CCBK, công suất 15W đa cổng - Hàng chính hãng': 'Energizer 10000mAh',
    'Pin sạc dự phòng MicroPack POWER STORE 10S 10000mAh - Hàng Chính Hãng': 'MicroPack POWER STORE 10S 10000mAh',
    'Sạc dự phòng 50.000mAh trong suốt, có sạc nhanh PD 120W, kèm 4 cáp sạc, tích hợp đèn ngủ - FTAKY Hàng Nhập Khẩu':'FTAKY 50000mAh',
    'Sạc Dự Phòng 50.000MAH Trong Suốt kèm cáp sạc nhanh 4 đầu, tích hợp 4 cáp sạc, làm đèn ngủ - FTAKY Hàng Nhập Khẩu': 'FTAKY 50000mAh',
    'Sạc dự phòng Energizer 30,000mAh /3.7V Li-Polymer UE30055PQBK, bảo hàng 24 tháng 1 đổi 1 - Hàng chính hãng': 'Energizer 30000mAh',
    'Sạc dự phòng Energizer 20,000mAh/3.7V Li-Polymer UE20050C - Hàng chính hãng bảo hành 2 năm 1 đổi 1': 'Energizer 20000mAh',
    'Pin Sạc Dự Phòng Trong Suốt Dung Lượng 10000mAh Linrui LR016 Đèn LED 3 Chế Độ - Hàng Chính Hãng':'Linrui LR016 10000mah',
    'Pin Sạc Dự Phòng Linrui LR010 Dung Lượng Chuẩn 20000mAh Kèm Cáp Sạc Sẵn, Màn Hình LED Hiển Thị Pin - Hàng Chính Hãng':'Linrui LR010 20000mah',
    'Pin Sạc Dự Phòng 20000mAh Linrui LR009 Sạc Nhanh, Cổng Ra Type C - Hàng Chính Hãng': 'Linrui LR009 20000mah',
    'Ugreen 130W PB721': 'Ugreen PB721',
    'Pin sạc dự phòng Samsung EB-P4520X 20000 mAh 45W-hàng chính hãng':'Samsung EB-P4520X 20000 mAh',
    'Xmobile 22.5W SnapGo GS-M3 kèm Cáp Lightning và Type C': 'Xmobile SnapGo GS-M3',
    'Baseus PicoGo 45W PPPG-1W45C kèm Cáp Type C': 'Baseus PicoGo PPPG-1W45C',
    'Xmobile 22.5W YM-672':'Xmobile YM-672',
    'Baseus Lipow 22.5W PPJP312 kèm Cáp Lightning và Type C': 'Baseus Lipow PPJP312',
    'Xmobile 22.5W A16': 'Xmobile A16',
    'Xmobile 22.5W H19 kèm Cáp Lightning và Type C': 'Xmobile H19',
    'Anker Zolo 165W A1695 kèm cáp dây rút Type C': 'Anker Zolo A1695',
    'Baseus Lipow 22.5W PPJP312 kèm Cáp Lightning và Type C': 'Baseus Lipow PPJP312',
    'Pin dự phòng sạc nhanh vỏ kim loại Baseus Adaman Metal Digital Display Quick Charge Power Bank (10.000mAh/20.000mAh, 22.5W, PD/QC3.0 Quick charge Power Bank)- Hàng chính hãng': 'Baseus Adaman Metal Digital 10000mAh 20000mAh',
    'Pin Sạc Dự Phòng Anker 323 1A1C 12W 10000mah USB-A và Type-C A1334 - Hàng Chính Hãng':'Anker 323 1A1C 10000mah',
    'Pin Sạc Dự Phòng 20000mAh RAVPower RP-PB1214 Sạc Nhanh 20W, Sạc Không Dây Nam Châm Tối Đa 15W, Tương Thích Mag.Safe iPhone - Hàng Chính Hãng':'RAVPower RP-PB1214 20000 mah',
    ' Pin Sạc Dự Phòng Không Dây Từ Tính Remax RPP-519 20000mAh Tích Hợp Kèm 2 Cáp Sạc Nhanh PD 22.5W - Hàng Chính Hãng Remax':'Remax RPP-519 20000mAh',
    'Sạc dự phòng Anker 737 PowerCore 1USB 2Type-C 140W PD3.1 24000mAh A1289 - Hàng chính hãng': 'Anker 737 24000mAh',
    'Sạc dự phòng Anker 533 PowerCore 1USB 2Type-C 30W 10000mAh A1256 - Hàng chính hãng': ' Anker 533 10000mAh',
    'Sạc dự phòng Anker 323 PowerCore PIQ 1USB 1Type-C 12W 10000mAh A1334 - Hàng chính hãng':' Anker 323 PIQ 10000mah',
    'Pin sạc dự phòng Rockspace Y11 sạc nhanh 22.5w dung lượng 20000mAh có tích hợp kèm dây sạc - Hàng chính hãng bảo hành 12 tháng':'Rockspace Y11 20000mah',
    'Sạc dự phòng Anker Nano Power Bank 1USB 1Type-C 30W 10000 mAh A1259 - Hàng chính hãng':'Anker Nano 10000mah',
    'Pin sạc dự phòng xiaomi 200000mah gen 4 22.5W model PB2022ZM - Hàng nhập khẩu':'xiaomi gen 4 200000mah',
    'Pin Sạc Dự Phòng Năng Lượng Mặt Trời Linrui LR001 10000mAh Tích Hợp Cáp Sạc Sẵn - Hàng Chính Hãng':'Linrui LR001 10000mAh',
    'Sạc dự phòng Energizer PD65W 27,000mAh/3.7V Li-ion XP27001PD, hỗ trợ sạc cho Laptop, chống cháy nổ - Hàng chính hãng bảo hành 12 tháng':'Energizer XP27001PD 27000mAh',
    'Sạc dự phòng Energizer 4,600mAh 3.6V Li-ion UE4600PQBK,Sạc nhanh PD 22.5W, bảo hành 24 tháng 1 đổi 1 - Hàng chính hãng': 'Energizer UE4600PQBK 4600mah',
    '[Mỹ] Sạc dự phòng Energizer QM5000 mini Sạc nam châm từ tính Dung lượng 5000mAh - Hàng chính hãng':'Energizer QM5000 5000mah',
    'Pin Sạc Dự Phòng Laptop HubbFast LAP100 100W 26800mAh, Sạc Nhanh PD Type-C USB-A, Màn Hình LED, Đa Năng, Hàng chính hãng': 'HubbFast LAP100 268000mah',
    'Sạc Dự Phòng Xiaomi Power Bank 10000mAh 22.5W Lite TH | Sạc Nhanh | Sạc Cùng Lúc Nhiều Thiết Bị | Bảo Hành 12 Tháng | Hàng Chính Hãng ':'Xiaomi Lite TH 10000mah',
    'Pin sạc dự phòng 20000mAh Type C PD QC 3.0 30W Anker A1384 kèm Cáp Type C - Hàng Chính Hãng': 'Anker A1384 20000mah',
    'Pin Sạc Dự Phòng Anker PowerCore 20100mAh - A1271': 'Anker A1271 20100mah',
    'Pin Sạc Dự Phòng LINRUI LR012 Dung Lượng 2600mAh Nhỏ Gọn - Hàng Chính Hãng':'LINRUI LR012 2600mah',
    'Sạc dự phòng ANKER PowerCore II 10000mAh - A1230':'ANKER PowerCore II A1230 10000mah',
    'Pin Sạc Dự Phòng Hoco Sạc Nhanh J87A PD20W QC3.0 20000Mah 20W Hàng Nhập Khẩu': 'Hoco J87A 20000mah' 
}
# Tạo cột mới 'products' bằng cách ánh xạ product_name sang name_map
df.loc[mask_powerbank, 'product_name'] = df.loc[mask_powerbank, 'product_name'].map(name_map)

In [105]:
df_watch = df[mask_powerbank].copy()

***Điện thoại***

In [106]:
#Điện thoại

***Đồng hồ thông minh***

In [107]:
mask_smartwatch = df['category'].str.contains('đồng hồ thông minh', case=False, na=False)

In [108]:
name_map = {
    'Đồng hồ thông minh Samsung Galaxy Watch 6 dành cho nam 2023 có theo dõi  Amoled Luôn hiển thị nhịp tim & theo dõi huyết áp Dây thép không gỉ': 'Samsung Galaxy Watch 6 2023 dây thép',
    ' Đồng Hồ Thông Minh Trẻ em AMA Q12S Model mới nhất ra mắt tháng 12/2021 - Hàng nhập khẩu': 'AMA Q12S',
    'Đồng hồ Thông minh Trẻ em AMA Watch Q15 lắp SIM nghe gọi 2 chiều 3 màu Xanh Hồng Đen Hàng chinh hãng': 'AMA Watch Q15',
    'Đồng hồ Thông minh Mini có Rung Gắn sim Định vị GPS, Wifi Đo huyết áp Nhịp tim nhắc nhở Vận động gọi Video call AMA Watch FA91S - Hàng nhập khẩu': 'AMA Watch FA91S',
    'Đồng hồ Thông minh Trẻ em Học sinh Tiểu học THCS THPT Chống nước Lắp sim Định vị Kép Smart Watch AMA HW13 Hàng nhập khẩu': 'AMA HW13',
    'Đồng hồ Thông minh Lắp Sim Định vị GPS Google kết nối Wifi 4G Blueltooth xem được Video Youtube TikTok tải App FB Messenger Skype WhatsApp Viber Wechat Kakao Line Tele chơi Game AMA Smart watch TK Android 8.1 cho Trẻ em Người lớn Hàng nhập khẩu': 'AMA TK Watch Android 8.1',
    'Apple Watch SE 2024 GPS (Viền nhôm, dây cao su)': 'Apple Watch SE 2024 viền nhôm dây cao su',
    'Vòng Đeo Tay Thông Minh Xiaomi Smart Band 9 Active – Hàng Chính Hãng': 'Xiaomi Smart Band 9 Active',
    'Garmin Forerunner 570 47mm dây silicone': 'Garmin Forerunner 570 47mm',
    'Amazfit Active 2 Square Sapphire 43.3mm dây daMẫu mới': 'Amazfit Active 2 Square 43.3mm dây da',
    'Garmin Forerunner 570 42.4mm dây silicone': 'Garmin Forerunner 570 42.4mm',
    'Đồng Hồ Thông Minh Xiaomi Redmi Watch 5 ACTIVE – Hàng Chính Hãng': 'Xiaomi Redmi Watch 5 Active',
    'Vòng Đeo Tay Thông Minh Xiaomi Mi Band 10 - Bản Quốc Tế - Hàng Chính Hãng': 'Xiaomi Mi Band 10',
    'Apple Watch Series 10 GPS (Viền nhôm, dây cao su)': 'Apple Watch Series 10 viền nhôm dây cao su',
    'Samsung Galaxy Watch Ultra LTE 47mm (2025) dây silicone': 'Samsung Galaxy Watch Ultra 47mm dây silicone',
    'Đồng Hồ Thông Minh Trẻ Em MAI DUY Y85S, Lắp Sim Nghe Gọi - Định Vị Cảnh Báo Nguy Hiểm, Chống Nước  -Hàng Nhập Khẩu': 'MAI DUY Y85S',
    'Đồng Hồ Thông Minh Trẻ Em AMA Watch Q12B Hàng nhập khẩu': 'AMA Watch Q12B',
    'Huawei Watch 5 46mm dây cao su': 'Huawei Watch 5 46mm dây cao su',
    'Samsung Galaxy Watch8 LTE 40mm dây siliconeMẫu mới': 'Samsung Galaxy Watch8 40mm dây silicone',
    'Huawei Watch 5 46mm dây compositeMẫu mới': 'Huawei Watch 5 46mm dây composite',
    'Garmin Venu X1 Sapphire 51.2mm dây nylonMẫu mới': 'Garmin Venu X1 51.2mm dây nylon',
    'Garmin Forerunner 570 47mm dây silicone': 'Garmin Forerunner 570 47mm dây silicone',
    'Amazfit Active 2 Square Sapphire 43.3mm dây daMẫu mới': 'Amazfit Active 2 Square 43.3mm dây da',
    'Garmin Forerunner 570 42.4mm dây silicone': 'Garmin Forerunner 570 42.4mm',
    'Samsung Galaxy Watch Ultra LTE 47mm (2025) dây silicone': 'Samsung Galaxy Watch Ultra 47mm dây silicone',
    'Samsung Galaxy Watch8 LTE 40mm dây siliconeMẫu mới': 'Samsung Galaxy Watch8 40mm dây silicone',
    'Đồng hồ Thông minh gắn Sim nghe gọi Chống nước IP67 Gọi Video call Định vị GPS, WIfi dàn cho Trẻ em học sinh cấp 1 2 5 6 7 8 9 10 11 12 tuổi AMA Watch D32 Hàng nhập khẩu': 'AMA Watch D32',
    'Đồng Hồ Định Vị Trẻ Em Wonlex KT18 - Hàng Chính Hãng': 'Wonlex KT18',
    'Đồng hồ thông minh Samsung Galaxy Watch5 40mm SM-R900N - Hàng chính hãng': 'Samsung Galaxy Watch5 SM-R900N 40mm',
    'Đồng hồ Thông minh Chống nước IP67 Dành cho Trẻ em nhuẩn AMA Watch Q16S Hàng chính hãng': 'AMA Watch Q16S',
    'Đồng Hồ Thông Minh cho Trẻ em 6 tuổi, 7 tuổi, 8 tuổi, 9 tuổi, 10 tuổi, 11 tuổi Lắp Sim Độc lập nghe gọi 2 chiều AMA Watch Y92 Chống nước Định vị Wifi độ chính xác cao Hàng nhập khẩu': 'AMA Watch Y92',
    'Đồng Hồ Thông Minh AMA Watch Nam Nữ chống nước | Gọi điện thoại Xem tin nhắn Theo dõi Sức khỏe vận động Thiết kế nhỏ gọn Thời trang Hàng nhập khẩu': 'AMA Watch D06',
    'Đồng Hồ Thông Minh Nữ AMA Watch S27 Thời trang Kết nối Bluetooth Theo dõi Sức khỏe Vận động Màn hình tròn Hàng chính hãng': 'AMA Watch S27'
}

# Tạo cột mới 'products' bằng cách ánh xạ product_name sang name_map
df.loc[mask_smartwatch, 'product_name'] = df.loc[mask_smartwatch, 'product_name'].map(name_map)

In [109]:
df_watch = df[mask_smartwatch].copy()

In [110]:
# Danh sách thương hiệu phổ biến
brands = ['AMA', 'Xiaomi', 'Apple', 'Samsung', 'Garmin', 'Huawei', 'Amazfit', 'Wonlex', 'MAI DUY']

# Hàm tách brand và model, xử lý riêng với AMA
def split_brand_model(product):
    for brand in brands:
        if product.startswith(brand):
            model = product[len(brand):].strip()
            if brand == "AMA" and model.startswith("Watch"):
                model = model[len("Watch"):].strip()
            return pd.Series([brand, model])
    return pd.Series(['Unknown', product])

# Áp dụng
df_watch[['brand', 'model']] = df_watch['product_name'].apply(split_brand_model)

# Gán ngược vào df chính
df.loc[mask_smartwatch, 'brand'] = df_watch['brand'].values
df.loc[mask_smartwatch, 'model'] = df_watch['model'].values

In [111]:
import re

In [112]:
info_patterns = r'\d+\.?\d*mm|viền [a-zA-ZÀ-ỹ\s]+|dây [a-zA-ZÀ-ỹ\s]+'

def split_model_info(text):
    # Tìm các phần info
    infos = re.findall(info_patterns, text, flags=re.IGNORECASE)
    info_str = ' '.join(infos).strip()

    # Loại bỏ info khỏi model
    model_cleaned = text
    for i in infos:
        model_cleaned = model_cleaned.replace(i, '')

    # Xoá khoảng trắng thừa
    model_cleaned = re.sub(r'\s{2,}', ' ', model_cleaned).strip()

    return pd.Series([model_cleaned, info_str])

In [113]:
df[['model', 'info']] = df['model'].fillna('').apply(split_model_info)

# Lưu file

In [114]:
df.to_excel("data_cleaned_2.xlsx", index=False)